In [6]:
import pandas as pd

tjh = pd.read_excel("./dataset/tjh/tjh_clincial.xlsx")

data = tjh.loc[:, ["name", "Stage"]]

# 更换列名
data.columns = ["name", "class"]
print(data)
data.to_csv("./dataset/tjh/tjh.csv", index=False)

        name  class
0    TJH0008      0
1    TJH0284      0
2    TJH0218      0
3    TJH0007      0
4    TJH0276      0
..       ...    ...
423  TJH0743      0
424  TJH0826      0
425  TJH0598      1
426  TJH0735      1
427  TJH0626      1

[428 rows x 2 columns]


In [27]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from skimage.measure import regionprops, label


# 计算肾脏的尺寸
dataset_list = ["tjh"]
for dataset in dataset_list:

    datapath = "./dataset/{}/origin_label".format(dataset)
    
    for name in os.listdir(datapath):
        if os.path.exists("./dataset/{}/new_label/{}".format(dataset, name)):
            continue
        # print("dataset: {}, name: {}".format(dataset, name))
        seg = sitk.ReadImage(os.path.join(datapath, name))
        label_array = sitk.GetArrayFromImage(seg)
        # print(label_array.shape)
        # assert label_array.shape[1] == 512
        if label_array.shape[1] != label_array.shape[2]:
            print("dataset: {}, name: {}".format(dataset, name))

        label_array1 = np.zeros_like(label_array)
        label_array2 = np.zeros_like(label_array)

        label_array1[label_array != 0] = 1
        label_array2[label_array != 2] = 0
        label_array2[label_array == 2] = 1

        # 找到最大的肿瘤连通域
        label_data2 = label(label_array2)
        region2 = regionprops(label_data2)
        region2 = sorted(region2, key=lambda x: x.area, reverse=True)
        tumor_array = np.zeros_like(label_array)
        tumor_array[region2[0].coords[:, 0], region2[0].coords[:, 1], region2[0].coords[:, 2]] = 1

        # 查看tumor_array在label_array1中的位置
        label_data1 = label(label_array1)
        region1 = regionprops(label_data1)
        region1 = sorted(region1, key=lambda x: x.area, reverse=True)
        
        # 查找region2[0]在region1中的位置,包含关系

        for region in region1:
            kidney_array = np.zeros_like(label_array)
            # 查看包含关系
            kidney_array[region.coords[:, 0], region.coords[:, 1], region.coords[:, 2]] = 1
            if np.sum(np.logical_and(kidney_array, tumor_array)) == np.sum(tumor_array):
                break
        
        kidney_array[tumor_array == 1] = 2
        new_label = sitk.GetImageFromArray(kidney_array)
        new_label.CopyInformation(seg)
        if not os.path.exists("./dataset/{}/new_label".format(dataset)):
            os.makedirs("./dataset/{}/new_label".format(dataset))

        sitk.WriteImage(new_label, "./dataset/{}/new_label/{}".format(dataset, name))
        # break


In [28]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from skimage.measure import regionprops, label

# 计算肾脏的尺寸
dataset_list = ["tjh"]
# dataset_list = ["tongji", "tjh", "henan", "xiangyang", "kits"]
for dataset in dataset_list:
    datapath = "./dataset/{}/origin_image".format(dataset)
    labelpath = "./dataset/{}/new_label".format(dataset)
    fatpath = "./dataset/{}/fat".format(dataset)
    crop_image_path = "./dataset/{}/crop_image".format(dataset)
    crop_label_path = "./dataset/{}/crop_label".format(dataset)
    if not os.path.exists(crop_image_path):
        os.makedirs(crop_image_path)
    if not os.path.exists(crop_label_path):
        os.makedirs(crop_label_path)

    for name in os.listdir(datapath):
        if os.path.exists(os.path.join(crop_image_path, name)):
            continue

        image = sitk.ReadImage(os.path.join(datapath, name))
        seg = sitk.ReadImage(os.path.join(labelpath, name))
        fat = sitk.ReadImage(os.path.join(fatpath, name))   

        # Calculate new size
        original_size = image.GetSize()
        original_spacing = image.GetSpacing()
        new_spacing = [0.5, 0.5, 1]  # 目标分辨率
        new_size = [
            int(round(original_size[0] * original_spacing[0] / new_spacing[0])),
            int(round(original_size[1] * original_spacing[1] / new_spacing[1])),
            int(round(original_size[2] * original_spacing[2] / new_spacing[2]))
        ]
        
        # 设置重采样器
        resampler = sitk.ResampleImageFilter()
        resampler.SetOutputSpacing(new_spacing)
        resampler.SetSize(new_size)
        resampler.SetOutputOrigin(image.GetOrigin())
        resampler.SetOutputDirection(image.GetDirection())
        
        # 对image进行B-spline插值重采样
        resampler.SetInterpolator(sitk.sitkBSpline3)
        new_image = resampler.Execute(image)
        
        # 对label进行最近邻插值重采样
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        new_seg = resampler.Execute(seg)
        new_fat = resampler.Execute(fat)
        label_array = sitk.GetArrayFromImage(new_seg)
        kidney_array = np.zeros_like(label_array)
        kidney_array[label_array >= 1] = 1
        x, y, z = np.where(kidney_array == 1)
        min_x = max(np.min(x)-30, 1)
        max_x = min(np.max(x)+30, kidney_array.shape[0]-2)      
        fat_array = sitk.GetArrayFromImage(new_fat)

        fat_array[fat_array != 4] = 0
        fat_array[fat_array == 4] = 3
        fat_array[label_array == 2] = 2
        fat_array[label_array == 1] = 1

        crop_image = new_image[:, :, min_x:max_x]
        crop_image.SetOrigin([0, 0, 0])

        crop_seg = fat_array[min_x:max_x, :, :]
        crop_seg = sitk.GetImageFromArray(crop_seg)
        crop_seg.SetDirection(new_image.GetDirection())
        crop_seg.SetSpacing(new_image.GetSpacing())
        crop_seg.SetOrigin([0, 0, 0])
        
        sitk.WriteImage(crop_image, os.path.join(crop_image_path, name))
        sitk.WriteImage(crop_seg, os.path.join(crop_label_path, name))

In [42]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from skimage.measure import regionprops, label

# 计算肾脏的尺寸
dataset_list = ["new"]
# dataset_list = ["tongji", "tjh", "henan", "xiangyang", "kits"]

for dataset in dataset_list:

    imagepath = "./dataset/{}/crop_image".format(dataset)
    labelpath = "./dataset/{}/crop_label".format(dataset)
    save_imagepath = "./dataset/{}/np_image_all".format(dataset)
    save_labelpath = "./dataset/{}/np_label_all".format(dataset)
    if not os.path.exists(save_imagepath):
        os.makedirs(save_imagepath)
    if not os.path.exists(save_labelpath):
        os.makedirs(save_labelpath)
    for name in os.listdir(imagepath):
        if os.path.exists(os.path.join(save_labelpath, name.split(".")[0] + ".npy")):
            continue
                          
        image = sitk.ReadImage(os.path.join(imagepath, name))
        seg = sitk.ReadImage(os.path.join(labelpath, name))
        # print(seg.GetDirection())
        image_array = sitk.GetArrayFromImage(image)
        label_array = sitk.GetArrayFromImage(seg)
        # label_array[label_array != 2] = 0
        # label_array[label_array == 2] = 1
        x, y, z = np.where(label_array == 2)
        x_min = max(0, np.min(x) - 10)
        x_max = min(label_array.shape[0]-1, np.max(x) + 10)
        # 从x_min到x_max中平均选择32个整数点
        # x_points = np.linspace(x_min, x_max, 32, dtype=int)
        
        # 中位数
        y_median = int((np.max(y) + np.min(y)) // 2)
        z_median = int((np.max(z) + np.min(z)) // 2)

        # 创建一个列表来存储所有切片
        # crop_labels = label_array[x_points, int(y_median - 112):int(y_median + 112), int(z_median - 112):int(z_median + 112)]
        # crop_images = image_array[x_points, int(y_median - 112):int(y_median + 112), int(z_median - 112):int(z_median + 112)]
        crop_labels = label_array[x_min:x_max, int(y_median - 112):int(y_median + 112), int(z_median - 112):int(z_median + 112)]
        crop_images = image_array[x_min:x_max, int(y_median - 112):int(y_median + 112), int(z_median - 112):int(z_median + 112)]
        # 转为int16
        crop_images = crop_images.astype(np.int16)
        # crop_labels = crop_labels.astype(np.int16)
        # print(type(crop_images))
        # # 查看数据类型
        # print(crop_labels.dtype)
        # 保存图像和标签
        np.save(os.path.join(save_imagepath, name.split(".")[0] + ".npy"), crop_images)
        np.save(os.path.join(save_labelpath, name.split(".")[0] + ".npy"), crop_labels)

In [52]:
import numpy as np

def pad_or_crop_to_target(image_array, label_array, target_depth=128, target_size=224):
    """
    调整图像和标签数组到指定的深度
    
    Parameters:
    image_array: shape (n, target_size, target_size) 的图像数组
    label_array: shape (n, target_size, target_size) 的标签数组
    target_depth: 目标深度 (默认128)
    target_size: 目标空间尺寸 (默认224)
    
    Returns:
    处理后的图像和标签数组，shape为(target_depth, target_size, target_size)
    """
    current_depth = image_array.shape[0]
    
    if current_depth < target_depth:
        # 需要填充
        pad_size = target_depth - current_depth
        pad_before = pad_size // 2
        pad_after = pad_size - pad_before
        
        # 对图像进行填充
        padded_image = np.pad(
            image_array,
            ((pad_before, pad_after), (0, 0), (0, 0)),
            mode='constant',
            constant_values=0
        )
        
        # 对标签进行填充
        padded_label = np.pad(
            label_array,
            ((pad_before, pad_after), (0, 0), (0, 0)),
            mode='constant',
            constant_values=0
        )
        
        return padded_image, padded_label
        
    elif current_depth > target_depth:
        # 需要裁剪
        start_idx = (current_depth - target_depth) // 2
        end_idx = start_idx + target_depth
        
        # 截取中间目标层数
        cropped_image = image_array[start_idx:end_idx, :, :]
        cropped_label = label_array[start_idx:end_idx, :, :]
        
        return cropped_image, cropped_label
        
    else:
        # 正好等于目标深度，直接返回
        return image_array, label_array

def process_and_save_data(input_image_path, input_label_path, output_image_path, output_label_path, 
                         target_depth=128, target_size=224):
    """
    处理并保存数据
    
    Parameters:
    input_image_path: 输入图像的numpy文件路径
    input_label_path: 输入标签的numpy文件路径
    output_image_path: 输出图像的numpy文件路径
    output_label_path: 输出标签的numpy文件路径
    target_depth: 目标深度 (默认128)
    target_size: 目标空间尺寸 (默认224)
    """
    # 加载数据
    image_array = np.load(input_image_path)
    label_array = np.load(input_label_path)
    
    # 检查输入维度
    assert image_array.shape[1] == target_size and image_array.shape[2] == target_size, \
        f"图像尺寸必须是{target_size}x{target_size}"
    assert label_array.shape[1] == target_size and label_array.shape[2] == target_size, \
        f"标签尺寸必须是{target_size}x{target_size}"
    assert image_array.shape == label_array.shape, "图像和标签尺寸必须相同"
    
    # 处理数据
    processed_image, processed_label = pad_or_crop_to_target(
        image_array, 
        label_array, 
        target_depth=target_depth,
        target_size=target_size
    )
    
    # 保存处理后的数据
    np.save(output_image_path, processed_image)
    np.save(output_label_path, processed_label)
    
    print(f"处理前形状: {image_array.shape}")
    print(f"处理后形状: {processed_image.shape}")


# 计算肾脏的尺寸
dataset_list = ["new", "henan", "kits", "xiangyang"]

for dataset in dataset_list:
    imagepath = "./dataset/{}/np_image_all".format(dataset)
    labelpath = "./dataset/{}/np_label_all".format(dataset)
    save_imagepath = "./dataset/{}/np_image_128".format(dataset)
    save_labelpath = "./dataset/{}/np_label_128".format(dataset)
    
    if not os.path.exists(save_imagepath):
        os.makedirs(save_imagepath)
    if not os.path.exists(save_labelpath):
        os.makedirs(save_labelpath)
    for name in os.listdir(imagepath):
        if os.path.exists(os.path.join(save_imagepath, name)):
            continue
        process_and_save_data(os.path.join(imagepath, name), os.path.join(labelpath, name), os.path.join(save_imagepath, name), os.path.join(save_labelpath, name), 
                              target_depth=128, target_size=224)
    

处理前形状: (103, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (71, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (77, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (79, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (62, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (91, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (89, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (79, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (75, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (55, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (37, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (33, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (69, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (73, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (70, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (75, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (69, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (38, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (67, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (57, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (57, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (57, 224, 224)
处理后形状: (128, 224, 224)
处理前形状: (4

In [ ]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from skimage.measure import regionprops, label

# 计算肾脏的尺寸

dataset_list = ["new", "henan", "kits", "xiangyang"]

result = pd.DataFrame(columns=['dataset', 'name', 'x', 'y', 'z', 'slce_num', 'gt', 'exclusion'])
for dataset in dataset_list:
    # dataset = 'tjh'
    save_labelpath = "./dataset/{}/np_label_all".format(dataset)
    labelpath = "./dataset/{}/{}.csv".format(dataset, dataset)
    label_df = pd.read_csv(labelpath)

    for name in os.listdir(save_labelpath):
        print(name)
        # name = 'TJH0251.npy'
        np_label = np.load(os.path.join(save_labelpath, name))
        # print(label.shape)
        np_label[np_label != 2] = 0
        np_label[np_label == 2] = 1
        # 使用 label 函数计算连通域
        labeled = label(np_label)
        region = regionprops(labeled)
        # 选择最大的连通域
        assert len(region) == 1, "More than one kidney found in the image, LABEL_NUM:{}".format(len(region))
        max_region = region[0]
        flag = 0
        if max_region.bbox[3] - max_region.bbox[0] >= 30 or max_region.bbox[4] - max_region.bbox[1] >= 60 or max_region.bbox[5] - max_region.bbox[2] >= 60:
            flag = 1
        if max_region.bbox[3] - max_region.bbox[0] > 100 or max_region.bbox[4] - max_region.bbox[1] > 200 or max_region.bbox[5] - max_region.bbox[2] > 200:
            flag = 2
        result_df = pd.DataFrame({
                                    'dataset': [dataset], 
                                    'name': [name], 
                                    'x': [max_region.bbox[3] - max_region.bbox[0]],
                                    'y': [max_region.bbox[4] - max_region.bbox[1]], 
                                    'z': [max_region.bbox[5] - max_region.bbox[2]],
                                    'slce_num': np_label.shape[0],
                                    'gt': [label_df[label_df['name'] == name.replace('.npy', '')]['class'].values[0]],
                                    'exclusion': [flag]
                                }, index=[0])

        result = pd.concat([result, result_df], ignore_index=True)
        print(result)
result.to_excel("size.xlsx", index=False)

In [49]:
import SimpleITK as sitk
import numpy as np
import os

def create_tumor_shell(input_mask_path, output_path, shell_width_mm=5.0):
    """
    为标签值为2的区域创建周围特定毫米宽度的环
    
    参数:
    input_mask_path: 输入多标签mask的nifti文件路径
    output_path: 输出环状mask的nifti文件路径
    shell_width_mm: 环的宽度，单位是毫米（默认5mm）
    """
    # 读取mask
    mask_img = sitk.ReadImage(input_mask_path)
    
    # 获取spacing信息
    spacing = mask_img.GetSpacing()
    
    # 计算每个方向上需要膨胀的体素数量
    voxel_radius = [int(np.ceil(shell_width_mm / sp)) for sp in spacing]
    print(f"Voxel radius: {voxel_radius}")
    
    # 将SimpleITK图像转换为numpy数组
    mask_arr = sitk.GetArrayFromImage(mask_img)
    
    # 只保留标签2，其他标签置为0
    tumor_mask = (mask_arr == 2).astype(np.uint8)
    
    # 转换为SimpleITK图像以使用膨胀函数
    img = sitk.GetImageFromArray(tumor_mask)
    img.CopyInformation(mask_img)
    
    # 创建膨胀滤波器
    dilate_filter = sitk.BinaryDilateImageFilter()
    dilate_filter.SetKernelRadius(voxel_radius)
    dilate_filter.SetForegroundValue(1)
    
    # 执行膨胀
    dilated = dilate_filter.Execute(img)
    
    # 转回numpy数组
    dilated_mask = sitk.GetArrayFromImage(dilated)
    
    # 计算环状区域（膨胀结果减去原始肿瘤区域）
    shell = (dilated_mask > 0) & (tumor_mask == 0)
    
    # 转换为SimpleITK图像并保持原始图像的信息
    shell = shell.astype(np.uint8)
    shell_img = sitk.GetImageFromArray(shell)
    shell_img.CopyInformation(mask_img)
    
    # 保存结果
    sitk.WriteImage(shell_img, output_path)
    
    return shell_img

# 计算肾肿瘤周围环，并保存
dataset_list = ["new", "henan", "kits", "xiangyang"]

for dataset in dataset_list:
    imagepath = f"./dataset/{dataset}/crop_image"
    labelpath = f"./dataset/{dataset}/crop_label"
    save_labelpath = f"./dataset/{dataset}/ring_5"
    
    # 确保输出目录存在
    os.makedirs(save_labelpath, exist_ok=True)

    for name in os.listdir(imagepath):
        inputPath = os.path.join(labelpath, name)
        outputPath = os.path.join(save_labelpath, name)
        
        try:
            create_tumor_shell(inputPath, outputPath, shell_width_mm=5.0)
            print(f"Successfully processed {name}")
        except Exception as e:
            print(f"Error processing {name}: {str(e)}")

Voxel radius: [10, 10, 5]
Successfully processed TJH0170.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed TJH0140.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed TJH0611.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed LIN RUI LAN.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed WANG GUO BIN.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed WU HUI XIANG.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed TJH0369.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed DAN BI ZHONG.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed WANG YING ZHEN.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed ZHAO BAO LIN.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed ZHENG AN BAO.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed PEI CHUN MEI.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed TJH0358.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed TJH0777.nii.gz
Voxel radius: [10, 10, 5]
Successfully processed LI JIN HUA.nii

In [ ]:
# 图像重采样
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
from skimage.measure import regionprops, label

dataset_list = ['henan', 'kits', 'tongji', 'xiangyang']
for dataset in dataset_list:
    origin_imagepath = './dataset/{}/origin_image'.format(dataset)
    origin_labelpath = './dataset/{}/origin_label'.format(dataset)
    new_imagepath = './dataset/{}/image_0.5'.format(dataset)
    new_labelpath = './dataset/{}/label_0.5'.format(dataset)
    # ring_path = './{}_dataset/ring'.format(dataset)
    
    # 如果新的路径不存在，则创建
    os.makedirs(new_imagepath, exist_ok=True)
    os.makedirs(new_labelpath, exist_ok=True)
    # os.makedirs(ring_path, exist_ok=True)

    # 目标spacing
    new_spacing = (0.5, 0.5, 0.5)

    # 获取文件名列表
    namelist = os.listdir(origin_imagepath)

    # 遍历每个文件
    for name in namelist:
        print(f"Processing {name} ...")
        if os.path.exists(os.path.join(new_imagepath, name)):
            continue

        # 读取原始图像和标签
        image = sitk.ReadImage(os.path.join(origin_imagepath, name))
        seg = sitk.ReadImage(os.path.join(origin_labelpath, name))

        label_array = sitk.GetArrayFromImage(seg)
        label_array[label_array != 2] = 0
        label_array[label_array == 2] = 1

        # 使用 label 函数计算连通域
        labeled = label(label_array)
        region = regionprops(labeled)

        # 选择最大的连通域
        max_region = max(region, key=lambda x: x.area)
        label_array = np.zeros_like(label_array)
        label_array[labeled == max_region.label] = 1

        # 断言只有一个连通域
        # assert len(region) == 1, "region number={}".format(len(region))

        seg = sitk.GetImageFromArray(label_array)
        seg.CopyInformation(image)

        # 获取原始的spacing和size
        original_spacing = image.GetSpacing()
        original_size = image.GetSize()
        
        # 计算新的size，保持物理尺寸不变
        new_size = [
            int(round(original_size[i] * (original_spacing[i] / new_spacing[i]))) 
            for i in range(3)
        ]
        print(original_size)
        print(original_spacing)
        print(new_size)
        # 设置重采样器
        resampler = sitk.ResampleImageFilter()
        resampler.SetOutputSpacing(new_spacing)
        resampler.SetSize(new_size)
        resampler.SetOutputOrigin(image.GetOrigin())
        resampler.SetOutputDirection(image.GetDirection())
        
        # 对image进行B-spline插值重采样
        resampler.SetInterpolator(sitk.sitkBSpline3)
        new_image = resampler.Execute(image)
        
        # 对label进行最近邻插值重采样
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        new_label = resampler.Execute(seg)

        # Convert to unsigned 8-bit integer
        new_label = sitk.Cast(new_label, sitk.sitkUInt8)
        
        # 保存处理后的图像和标签
        sitk.WriteImage(new_image, os.path.join(new_imagepath, name))
        sitk.WriteImage(new_label, os.path.join(new_labelpath, name.replace('.nii.gz', '_seg.nii.gz')))
    

In [ ]:
# 图像重采样
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
from skimage.measure import regionprops, label

dataset_list = ['henan', 'kits', 'tongji', 'xiangyang']
for dataset in dataset_list:
    origin_imagepath = './dataset/{}/origin_image'.format(dataset)
    origin_labelpath = './dataset/{}/origin_label'.format(dataset)
    new_imagepath = './dataset/{}/image'.format(dataset)
    new_labelpath = './dataset/{}/label'.format(dataset)
    ring_path = './dataset/{}/ring'.format(dataset)
    # 目标spacing
    new_spacing = (1, 1, 1)

    # 获取文件名列表
    namelist = os.listdir(origin_imagepath)

    # 遍历每个文件
    for name in namelist:
        print(f"Processing {name} ...")
        
        # 读取原始图像和标签
        image = sitk.ReadImage(os.path.join(origin_imagepath, name))
        seg = sitk.ReadImage(os.path.join(origin_labelpath, name))

        label_array = sitk.GetArrayFromImage(seg)
        label_array[label_array != 2] = 0
        label_array[label_array == 2] = 1

        # 使用 label 函数计算连通域
        labeled = label(label_array)
        region = regionprops(labeled)
        if len(region) == 1:
            continue
        else:
            # 选择最大的连通域
            max_region = max(region, key=lambda x: x.area)
            label_array = np.zeros_like(label_array)
            label_array[labeled == max_region.label] = 1


        seg = sitk.GetImageFromArray(label_array)
        seg.CopyInformation(image)
        
        # 获取原始的spacing和size
        original_spacing = image.GetSpacing()
        original_size = image.GetSize()
        
        # 计算新的size，保持物理尺寸不变
        new_size = [
            int(round(original_size[i] * (original_spacing[i] / new_spacing[i]))) 
            for i in range(3)
        ]
        print(original_size)
        print(original_spacing)
        print(new_size)
        # 设置重采样器
        resampler = sitk.ResampleImageFilter()
        resampler.SetOutputSpacing(new_spacing)
        resampler.SetSize(new_size)
        resampler.SetOutputOrigin(image.GetOrigin())
        resampler.SetOutputDirection(image.GetDirection())
        
        # 对image进行B-spline插值重采样
        resampler.SetInterpolator(sitk.sitkBSpline3)
        new_image = resampler.Execute(image)
        
        # 对label进行最近邻插值重采样
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        new_label = resampler.Execute(seg)

        # Convert to unsigned 8-bit integer
        new_label = sitk.Cast(new_label, sitk.sitkUInt8)
        
        # 保存处理后的图像和标签
        sitk.WriteImage(new_image, os.path.join(new_imagepath, name))
        sitk.WriteImage(new_label, os.path.join(new_labelpath, name.replace('.nii.gz', '_seg.nii.gz')))
        
        # print(f"Saved resampled image and label for {name}.")

        for dilation_radius in [3, 5, 7]:

            binary_filter = sitk.BinaryDilateImageFilter()
            binary_filter.SetKernelRadius(dilation_radius)  # 设置膨胀半径
            binary_filter.SetForegroundValue(1)  # 只对值为1的区域进行膨胀
            dilated_label = binary_filter.Execute(new_label)

            # 将膨胀后的区域减去原始的label区域，得到环形区域
            label_array = sitk.GetArrayFromImage(new_label)
            dilated_array = sitk.GetArrayFromImage(dilated_label)
            
            # 计算环形区域 (膨胀区域 - 原始肿瘤区域)
            ring_array = (dilated_array - label_array).clip(min=0)  # 确保值不小于0

            # 将环形区域转换回SimpleITK图像
            ring_image = sitk.GetImageFromArray(ring_array)
            ring_image.CopyInformation(new_label)  # 保持原始label图像的元数据信息

            # 保存为新的NIfTI文件
            sitk.WriteImage(ring_image, os.path.join(ring_path, name.replace('.nii.gz', '_ring{}.nii.gz'.format(dilation_radius))))


In [ ]:
# 计算肿瘤环，对image和label重采样为1，1，1
import SimpleITK as sitk
import os

# 文件路径定义
dataset = 'henan'
origin_imagepath = './{}_dataset/origin_image'.format(dataset)
origin_labelpath = './{}_dataset/origin_label'.format(dataset)
new_imagepath = './{}_dataset/image'.format(dataset)
new_labelpath = './{}_dataset/label'.format(dataset)
ring_path = './{}_dataset/ring'.format(dataset)

# 如果新的路径不存在，则创建
os.makedirs(new_imagepath, exist_ok=True)
os.makedirs(new_labelpath, exist_ok=True)
os.makedirs(ring_path, exist_ok=True)

# 目标spacing
new_spacing = (1, 1, 1)

# 获取文件名列表
namelist = os.listdir(origin_imagepath)

# 遍历每个文件
for name in namelist:
    print(f"Processing {name} ...")
    if os.path.exists(os.path.join(ring_path, name.replace('.nii.gz', '_ring7.nii.gz'))):
        continue
    
    # 读取原始图像和标签
    image = sitk.ReadImage(os.path.join(origin_imagepath, name))
    label = sitk.ReadImage(os.path.join(origin_labelpath, name))

    label_array = sitk.GetArrayFromImage(label)
    label_array[label_array != 2] = 0
    label_array[label_array == 2] = 1
    label = sitk.GetImageFromArray(label_array)
    label.CopyInformation(image)
    
    # 获取原始的spacing和size
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    
    # 计算新的size，保持物理尺寸不变
    new_size = [
        int(round(original_size[i] * (original_spacing[i] / new_spacing[i]))) 
        for i in range(3)
    ]
    print(original_size)
    print(original_spacing)
    print(new_size)
    # 设置重采样器
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    
    # 对image进行B-spline插值重采样
    resampler.SetInterpolator(sitk.sitkBSpline3)
    new_image = resampler.Execute(image)
    
    # 对label进行最近邻插值重采样
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    new_label = resampler.Execute(label)

    # Convert to unsigned 8-bit integer
    new_label = sitk.Cast(new_label, sitk.sitkUInt8)
    
    # 保存处理后的图像和标签
    sitk.WriteImage(new_image, os.path.join(new_imagepath, name))
    sitk.WriteImage(new_label, os.path.join(new_labelpath, name.replace('.nii.gz', '_seg.nii.gz')))
    
    # print(f"Saved resampled image and label for {name}.")

    for dilation_radius in [3, 5, 7]:

        binary_filter = sitk.BinaryDilateImageFilter()
        binary_filter.SetKernelRadius(dilation_radius)  # 设置膨胀半径
        binary_filter.SetForegroundValue(1)  # 只对值为1的区域进行膨胀
        dilated_label = binary_filter.Execute(new_label)

        # 将膨胀后的区域减去原始的label区域，得到环形区域
        label_array = sitk.GetArrayFromImage(new_label)
        dilated_array = sitk.GetArrayFromImage(dilated_label)
        
        # 计算环形区域 (膨胀区域 - 原始肿瘤区域)
        ring_array = (dilated_array - label_array).clip(min=0)  # 确保值不小于0

        # 将环形区域转换回SimpleITK图像
        ring_image = sitk.GetImageFromArray(ring_array)
        ring_image.CopyInformation(new_label)  # 保持原始label图像的元数据信息

        # 保存为新的NIfTI文件
        sitk.WriteImage(ring_image, os.path.join(ring_path, name.replace('.nii.gz', '_ring{}.nii.gz'.format(dilation_radius))))

In [ ]:
# 计算dl组学参数
